Reference: https://github.com/pytorch/fairseq/blob/main/examples/translation/README.md

In [ ]:
! git clone "https://github.com/pytorch/fairseq.git"

Cloning into 'fairseq'...
remote: Enumerating objects: 31205, done.
remote: Total 31205 (delta 0), reused 0 (delta 0), pack-reused 31205
Receiving objects: 100% (31205/31205), 21.46 MiB | 21.69 MiB/s, done.
Resolving deltas: 100% (23018/23018), done.


In [ ]:
% cd fairseq
! pip install .
! pip install sacremoses

/content/fairseq
Processing /content/fairseq
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 223 kB 25.6 MB/s 
     |████████████████████████████████| 123 kB 50.6 MB/s 
     |████████████████████████████████| 90 kB 9.5 MB/s 
     |████████████████████████████████| 112 kB 77.4 MB/s 
     |████████████████████████████████| 596 kB 62.4 MB/s 
  Created wheel for fairseq: filename=fairseq-1.0.0a0+806855b-cp37-cp37m-linux_x86_64.whl size=1614924

In [ ]:
% cd ./fairseq_cli/

[Errno 2] No such file or directory: './fairseq_cli/'
/content/fairseq/fairseq_cli


In [ ]:
% pwd

'/content/fairseq/fairseq_cli'

Upload min, ind (train, valid, test) inside fairseq_cli folder

In [ ]:
! python preprocess.py --source-lang jav --target-lang ind --trainpref train --validpref valid --testpref test --destdir /content/ --joined-dictionary

2022-04-21 00:30:50 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-04-21 00:30:50 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False

In [ ]:
#! rm -r checkpoints

In [ ]:
! CUDA_VISIBLE_DEVICES=0 fairseq-train \
    /content/ \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

Streaming output truncated to the last 5000 lines.
epoch 062 | valid on 'valid' subset:  38% 3/8 [00:03<00:06,  1.24s/it]2022-04-21 01:08:48 | INFO | fairseq.tasks.translation | example hypothesis: supaya aku ora bisa nggèndèng para wong kang padha mbangun-turut ana ing Yérusalèm, supaya bisa nggèndèng para wong suci ana ing kono,
2022-04-21 01:08:48 | INFO | fairseq.tasks.translation | example reference: supaya aku kauwalaké saka wong-wong ing Yudéa kang padha ora mbangun-miturut, lan supaya peladosanku kanggo Yérusalèm iku UNKNOWNTOKENINREF kanthi becik déning para suci ing kono,

epoch 062 | valid on 'valid' subset:  50% 4/8 [00:04<00:04,  1.24s/it]2022-04-21 01:08:49 | INFO | fairseq.tasks.translation | example hypothesis: Ana ing Panjenengané ngandika, "Iki pisungsung sarta kurban, Kurban obaran saha kurban mboten Paduka ndadosaken Paduka, lan sanadyan Paduka iku kang ndadèkaké angger-anggering Torèt -- sanadyan kang nindakaké angger-anggering Torèt.
2022-04-21 01

Save checkpoints to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/fairseq/fairseq_cli/checkpoints/checkpoint_best.pt /content/drive/MyDrive

Evaluate model

In [ ]:
! fairseq-generate /content/ \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

Streaming output truncated to the last 5000 lines.
D-753	-1.1210083961486816	Rahayu batur , kang padha ngalami bendarané nglakoni iku , nalika bendarané abdi-abdi mau teka ,
P-753	-0.0070 -1.3211 -0.1484 -0.1077 -1.1277 -0.1326 -1.8607 -2.4282 -1.0313 -0.0655 -0.0520 -0.0783 -5.1807 -0.6152 -3.7178 -0.1539 -1.0291
S-104	Ketika itu sudah dekat hari raya orang Yahudi , yaitu hari raya <unk> <unk>
T-104	Nalika iku ngarepaké riyayané wong Yahudi , yaiku riyaya <<unk>> <<unk>>
H-104	-0.9470048546791077	Nalika samana wus cedhak riyaya Paskahé wong Yahudi , yaiku dina kang aran Harmagédon.
D-104	-0.9470048546791077	Nalika samana wus cedhak riyaya Paskahé wong Yahudi , yaiku dina kang aran Harmagédon.
P-104	-0.1934 -0.6808 -0.2258 -0.1476 -0.1253 -0.2352 -0.1029 -0.0366 -0.1075 -0.0322 -0.2915 -1.2613 -5.0826 -5.5903 -0.0921
S-417	Apabila kamu <unk> , kamu dapat mengetahui dari padanya <unk> akan rahasia Kristus ,
T-417	Yèn kokwaca , kowé bisa nyumurupi <<unk>> bab wewadiné Kristus 